In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

%matplotlib inline

In [ ]:
# Load the dataset
data = pd.read_csv('path_to_your_dataset/fer2013.csv')

# Preprocess the data
def preprocess_input(x):
    x = np.array(x.split(), dtype=np.float32)
    x = x.reshape(48, 48, 1)
    x = x / 255.0
    return x

data['pixels'] = data['pixels'].apply(preprocess_input)
data['emotion'] = data['emotion'].astype(int)

# Split data into training and validation sets
train_data = data[data['Usage'] == 'Training']
val_data = data[data['Usage'] == 'PublicTest']

x_train = np.stack(train_data['pixels'].values)
y_train = train_data['emotion'].values
x_val = np.stack(val_data['pixels'].values)
y_val = val_data['emotion'].values

# Convert data to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(buffer_size=1024).batch(64)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(64)

In [ ]:
def dual_path_cnn(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)

    # Main path
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Residual path
    residual = layers.Conv2D(64, (1, 1), padding='same', activation='relu')(inputs)
    residual = layers.BatchNormalization()(residual)

    # Combine paths
    combined = layers.Add()([x, residual])
    combined = layers.ReLU()(combined)

    # Further layers
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(combined)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Flatten and dense layers
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model

# Define input shape and number of classes
input_shape = (48, 48, 1)
num_classes = 7
model = dual_path_cnn(input_shape, num_classes)

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_dataset, validation_data=val_dataset, epochs=50,
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])

# Save the model
model.save('fer_ducnn_model.h5')

In [ ]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(val_dataset)
print(f'Validation accuracy: {val_accuracy:.2f}')

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Model Loss')

plt.show()